In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from data_cleaning import *
import implementations as imp
import plots
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

COLUMN_TO_DROP = 22

# Import data

In [2]:
y_train, x_train, ids_train = load_csv_data("../data/train.csv")
y_test, x_test, ids_test = load_csv_data("../data/test.csv")

# Clean and add features

In [3]:
PRI_jet_num = np.array([x_train[:, COLUMN_TO_DROP]])
print(PRI_jet_num.shape)
del_x_train = np.delete(x_train, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

print()

replaced_x_train = replace_undefined(del_x_train, UNDEFINED_VALUE, 0.0)
poly_x_train = create_poly_features(replaced_x_train, 3)
print(poly_x_train[0][0])
print(poly_x_train.shape)

print()

norm_x_train, train_data_mean, train_data_std = mean_std_normalization(poly_x_train)
print(norm_x_train[0][0])
print(norm_x_train.shape)

print()

new_x_train = np.concatenate((norm_x_train, PRI_jet_num.T), axis=1)
print(new_x_train.shape)

(1, 250000)
(250000, 29)

138.47
(250000, 87)

0.5132216131919056
(250000, 87)

(250000, 88)


In [4]:
PRI_jet_num = np.array([x_test[:, COLUMN_TO_DROP]])
print(PRI_jet_num[0:5])
print(PRI_jet_num.shape)
del_x_test = np.delete(x_test, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

replaced_x_test = replace_undefined(del_x_test, UNDEFINED_VALUE, 0.0)
poly_x_test = create_poly_features(replaced_x_test, 3)
print(poly_x_test[0][0])
print(poly_x_test.shape)

norm_x_test, test_data_mean, test_data_std = mean_std_normalization(poly_x_test, train_data_mean, train_data_mean)
print(norm_x_test[0][0])
print(norm_x_test.shape)

new_x_test = np.concatenate((norm_x_test, PRI_jet_num.T), axis=1)
print(new_x_test.shape)

[[0. 1. 0. ... 0. 1. 0.]]
(1, 568238)
(250000, 29)
0.0
(568238, 87)
-1.0
(568238, 87)
(568238, 88)


# The model

In [ ]:
gamma = 0.1
max_iters = 100
initial_w = np.ones(new_x_train.shape[1], dtype=np.float64)

rg_loss, best_rg_weight = imp.logistic_regression(y_train, new_x_train, initial_w, max_iters, gamma)
print(len(rg_loss))

Loss in iteration 0: 1529113.584715708
Loss in iteration 1: 1516696794.1490726
Loss in iteration 2: -4689984083.985154
Loss in iteration 3: -9480558379.686684
Loss in iteration 4: -11743310644.382421
Loss in iteration 5: -13873699736.308298
Loss in iteration 6: -15984585251.409456
Loss in iteration 7: -18085845591.85376
Loss in iteration 8: -20182108324.636673
Loss in iteration 9: -22275704285.078957
Loss in iteration 10: -24367899747.224854
Loss in iteration 11: -26459055857.064762
Loss in iteration 12: -28549491527.841103
Loss in iteration 13: -30639440843.591393
Loss in iteration 14: -32729010307.34956
Loss in iteration 15: -34818313557.32939
Loss in iteration 16: -36907394512.21491
Loss in iteration 17: -38996306767.1004
Loss in iteration 18: -41085110094.8328
Loss in iteration 19: -43173784796.511505
Loss in iteration 20: -45262364789.53327
Loss in iteration 21: -47350870151.06121
Loss in iteration 22: -49439296135.25639
Loss in iteration 23: -51527670250.5036
Loss in iteration 24

In [ ]:
# Plot
plt.subplots(figsize=(20,10))
plt.plot(rg_loss)
plt.legend(["Training loss"])
plt.grid()
plt.title("Loss for Logistic Regression")
plt.xlabel("Iteration number")
plt.ylabel("MSE")
plt.show()

# Testing

In [ ]:
y_pred = predict_labels(best_rg_weight[-1], new_x_test)
n = len(y_pred)
correct = 0
for i in range(n):
    if (y_pred[i] == y_test[i]):
        correct += 1
print(str(correct) + " of " + str(n) + " correct, precentage: " + str(correct/n)) 